In [ ]:
from fastai.vision.all import *

In [ ]:
#from fastai import test_utils
#test_utils.show_install()

In [ ]:
torch.cuda.set_device(0)

In [ ]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()

In [ ]:
path = Path('../input/sorghum-id-fgvc-9'); path.ls()

In [ ]:
sorghum_df = pd.read_csv(path/'train_cultivar_mapping.csv');sorghum_df.head()

In [ ]:
path

In [ ]:
allowed_image = []
for i in (path/'train_images').ls():
    va = i.name in sorghum_df.image.to_list()
    if va:
        allowed_image.append(i.name)

In [ ]:
sorghum_df = sorghum_df[sorghum_df.image.isin(allowed_image)]

In [ ]:
sorghum_df.cultivar.value_counts()

In [ ]:
dls = ImageDataLoaders.from_df(sorghum_df, path/'train_images', valid_pct=0.10,
                                   item_tfms=Resize(128),
                                   bs=32, num_workers=4, label_col="cultivar")

In [ ]:
dls.show_batch()

In [ ]:
#print(dls.vocab)
print(dls.c)

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=[error_rate, accuracy], model_dir="/tmp/model/").to_fp16()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit(5)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix(figsize=(5,5), dpi=60)